# ChatGPT Sentiment Analysis of Facebook group comments

In [13]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Get Data from Facebook Group Posts

In [ ]:
import json
import requests

with open("../online-PT-social-media-NLP/notebooks/credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['am_ig_user_id']
access_token = credentials['am_fb_access_token']
group_id = credentials['SSC_group_id']

posts_data_dict = dict()
posts_df_dict = dict()
comments_response_dict = dict()
comments_df_dict = dict()
replies_dict = dict()

def get_facebook_posts(group_id, access_token, limit=30):
    """
    Makes a GET request to pull posts from a Facebook group using the Facebook API.
    Outputs the results into a DataFrame so that each post is a row along with its time stamp, message,
    permalink, comments count, reactions, and post ID.
    Also outputs the response.json().

    Parameters:
        group_id (str): The ID of the Facebook group to pull posts from.
        access_token (str): An access token with the appropriate permissions to access the group's content.
        limit (int): The number of posts to retrieve (default is 100).

    Returns:
        pandas.DataFrame: A DataFrame containing the posts and their associated data.
    """
    url = f"https://graph.facebook.com/{group_id}/feed"

    params = {
        "access_token": access_token,
        "fields": "created_time,message,permalink_url,comments.limit(10).summary(true),reactions.limit(500).summary(true).type(LIKE).as(like),reactions.limit(500).summary(true).type(LOVE).as(love),reactions.limit(500).summary(true).type(WOW).as(wow),reactions.limit(500).summary(true).type(HAHA).as(haha),reactions.limit(500).summary(true).type(SAD).as(sad),reactions.limit(500).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    response = requests.get(url, params=params)
    status_code = response.status_code
    print("Response Status Code:", status_code)
    data = response.json()
    return data

## I will use this as part of the final code for the task
def process_response(data):
    posts = []
    comments = []
    for post in data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post.get("wow", {}).get("summary", {}).get("total_count", 0), post.get("haha", {}).get("summary", {}).get("total_count", 0), post.get("sad", {}).get("summary", {}).get("total_count", 0), post.get("angry", {}).get("summary", {}).get("total_count", 0)
        post_id = post.get("id")
        posts.append({
            "post_id": post_id,
            "created_time": created_time,
            "message": message,
            "comments_count": comments_count,
            "like_count": reactions[0],
            "love_count": reactions[1],
            "wow_count": reactions[2],
            "haha_count": reactions[3],
            "sad_count": reactions[4],
            "angry_count": reactions[5],
            "permalink_url": permalink_url,
        })

    df = pd.DataFrame(posts)
    # Reorder columns
    df = df[["post_id", "created_time", "message", "comments_count", "like_count", "love_count", "wow_count", "haha_count", "sad_count", "angry_count", "permalink_url"]]
    
    return df

def get_fb_comments(group_id, access_token, limit=30):
    """
    Pulls comments from a Facebook group using the Facebook API and outputs the results into a pandas DataFrame.
    
    Parameters:
    group_id (str): The ID of the Facebook group to pull comments from.
    access_token (str): A Facebook access token with the necessary permissions to access the group.
    
    Returns:
    Tuple(pandas.DataFrame, dict): A tuple containing a DataFrame with each comment as a row along with its details and replies,
    and the full JSON response from the API.
    """
    # Set up the API endpoint and parameters
    url = f"https://graph.facebook.com/{group_id}/feed"
    params = {
        "fields": "comments{created_time,message,from,comments{created_time,message,from}}",
        "access_token": access_token,
        "limit": limit
    }

    # Make the GET request
    response = requests.get(url, params=params)
    
    # Convert the response to a DataFrame
    comments = []
    for post in response.json()["data"]:
        post_comments = post.get("comments", {}).get("data", [])
        for comment in post_comments:
            comment_dict = {
                "post_id": post.get("id"),
                "created_time": comment.get("created_time"),
                "message": comment.get("message"),
                "author_id": comment.get("from", {}).get("id"),
                "author_name": comment.get("from", {}).get("name"),
                "parent_id": None,
                "parent_name": None,
            }
            parent_comment = comment.get("comments", {}).get("data", [])
            if parent_comment:
                parent_comment = parent_comment[0]
                comment_dict["parent_id"] = parent_comment.get("id")
                comment_dict["parent_name"] = parent_comment.get("from", {}).get("name")
            comments.append(comment_dict)
    df = pd.DataFrame(comments)
    return df, response.json()

def create_reply_dataframe(json_object, add_replies=False):
    # Initialize empty lists to store data
    post_id_list = []
    parent_comment_message_list = []
    comment_id_list = []
    reply_comment_message_list = []
    reply_created_time_list = []
    reply_id_list = []
    
    # Loop through each post in the json object
    for post in json_object:
        # Get the post ID
        post_id = post['id']
        
        try: # SH 2023-03-22 10:50 Added this 
            # Loop through each comment in the post
            for comment in post['comments']['data']:
                # Get the comment ID and message
                comment_id = comment['id']
                parent_comment_message = comment['message']
                
                # Check if the comment has any replies
                if add_replies == True:
                    # Loop through each reply to the comment
                    for reply in comment['comments']['data']:
                        if 'comments' in comment:
                            # Get the reply ID, message, and creation time
                            reply_id = reply['id']
                            reply_comment_message = reply['message']
                            reply_created_time = reply['created_time']
                            
                            # Add the data to the lists
                            post_id_list.append(post_id)
                            parent_comment_message_list.append(parent_comment_message)
                            comment_id_list.append(comment_id)
                            reply_comment_message_list.append(reply_comment_message)
                            reply_created_time_list.append(reply_created_time)
                            reply_id_list.append(reply_id)
                else:
                    # Add the data to the lists
                    post_id_list.append(post_id)
                    parent_comment_message_list.append(parent_comment_message)
                    comment_id_list.append(comment_id)
        except Exception  as error:
            # exc_type, exc_obj, tb = sys.exc_info()
            # f = tb.tb_frame
            # lineno = tb.tb_lineno
            # filename = f.f_code.co_filename
            # print("An error occurred on line", lineno, "in", filename, ":", error)
            pass
    
    # Create a dataframe from the lists
    if add_replies == True:
        df = pd.DataFrame({
            'post_id': post_id_list,
            'parent_comment_message': parent_comment_message_list,
            'comment_id': comment_id_list,
            'reply_comment_message': reply_comment_message_list,
            'reply_created_time': reply_created_time_list,
            'reply_id': reply_id_list
        })
    else:
        df = pd.DataFrame({
            'post_id': post_id_list,
            'parent_comment_message': parent_comment_message_list,
            'comment_id': comment_id_list,
        })
    return df

def merge_dataframes(comments_df, posts_df, replies_df, add_replies=False):
    # merge comments and posts dataframes on post_id
    merged_df = pd.merge(comments_df, posts_df, on='post_id', how='outer')
    # merge replies dataframe on comment_id
    merged_df = pd.merge(merged_df, replies_df, left_on='message_x', right_on='parent_comment_message', how='left')
    if add_replies == True:
        # rename columns
        merged_df = merged_df.rename(columns={
            'post_id_x': 'post_id',
            'created_time_x': 'comment_created_time',
            'created_time_y': 'post_created_time',
            'message_x': 'comment_message',
            'message_y': 'post_message',
            'parent_name_x': 'parent_name',
            'reply_comment_message': 'reply_message',
            'reply_created_time': 'reply_created_time',
            'reply_id': 'reply_id'
        })
        # drop unnecessary columns
        merged_df = merged_df.drop(columns=['parent_name', 'parent_id'])
        # reorder columns
        merged_df = merged_df[['post_id', 'post_message', 'post_created_time', 'comment_created_time', 'parent_comment_message',
                            'comment_id', 'reply_message', 'reply_created_time', 'reply_id']]
    else:
        # rename columns
        merged_df = merged_df.rename(columns={
            'post_id_x': 'post_id',
            'created_time_x': 'comment_created_time',
            'created_time_y': 'post_created_time',
            'message_x': 'comment_message',
            'message_y': 'post_message',
            'parent_name_x': 'parent_name',
        })
        # drop unnecessary columns
        merged_df = merged_df.drop(columns=['parent_name', 'parent_id'])
        # reorder columns
        merged_df = merged_df[['post_id', 'post_message', 'post_created_time', 'comment_created_time', 'parent_comment_message',
                            'comment_id']]
    return merged_df

def process_posts(df):
    """
    Return the unique non-null parent_comment_message values and the associated post_message values.
    """
    df = df[['parent_comment_message', 'post_message']].drop_duplicates()
    df = df[df['parent_comment_message'].notnull()]
    df = df[df['post_message'].notnull()]
    results = pd.DataFrame(
        df.apply(lambda row: f"You: {row['post_message']}\n\nMember: {row['parent_comment_message']}", axis=1).unique(),
        columns=['text'])
    return results


iteration = 1.11

# Get Facebook group posts
posts_data_dict[iteration] = get_facebook_posts(group_id, access_token)
posts_df_dict[iteration] = process_response(posts_data_dict[iteration])
comments_df_dict[iteration], comments_response_dict[iteration] = get_fb_comments(group_id, access_token)
replies_dict[iteration] = create_reply_dataframe(comments_response_dict[iteration]['data'], add_replies=0)
merged = merge_dataframes(comments_df_dict[iteration], posts_df_dict[iteration], replies_dict[iteration])


In [14]:
merged

,post_id,post_message,post_created_time,comment_created_time,parent_comment_message,comment_id
0,2139238999669147_3433340976925603,Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?,2023-06-25T05:50:18+0000,2023-06-25T06:33:26+0000,Yes 😁,3433357203590647
1,2139238999669147_3433340976925603,Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?,2023-06-25T05:50:18+0000,2023-06-25T11:37:43+0000,I’m in Airlie beach too!!,3433488186910882
2,2139238999669147_3433340976925603,Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?,2023-06-25T05:50:18+0000,2023-06-25T22:36:44+0000,You are less than 2 hours away from where I am thats so cool I love airlie beach just never find the time to go there anymore,3433796753546692
3,2139238999669147_3432149000378134,How I stay bikini confident year round 👙\n\nFirst off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel.\n\nSo let me ...,2023-06-23T06:46:23+0000,2023-06-23T07:01:32+0000,…. And you look awesome 👏👏,3432154477044253
4,2139238999669147_3432149000378134,How I stay bikini confident year round 👙\n\nFirst off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel.\n\nSo let me ...,2023-06-23T06:46:23+0000,2023-06-23T09:39:41+0000,"Mind set is everything! Recently in Thailand, our last day. 12 hrs to kill before the flight, wanted to go for a swim, but didn’t want our baggage to weigh over the 25kg with wet swimmers (LOTS of...",3432216450371389
...,...,...,...,...,...,...
74,2139238999669147_3426735054252862,If you want to start seeing long term results - we need to stop focusing on exercise on a way to ‘burn calories’. Exercise will only ever contribute to a SMALL a...,2023-06-14T20:48:08+0000,NaN,NaN,NaN
75,2139238999669147_3425629184363449,Sarah - Nurse - 29 - Roscommon\n\n‘Before signing up to Amanda’s Strong and Sassy programme - I had attempted to write to Amanda numerous times but deleted the message’\n\n‘Don’t wait as long as I...,2023-06-13T06:42:27+0000,NaN,NaN,NaN
76,2139238999669147_3422437144682653,How to win the weekend 🏆Tag someone who needs to hear this 👌#nutrition #nutritiongoals #onlinecoach #fitness #fitnesscoach #realresults #realnutrition #explore #explorepage,2023-06-08T20:49:45+0000,NaN,NaN,NaN
77,2139238999669147_3419678831625151,"Perfectionist ladies, it's time to ditch the all-or-nothing mentality and adopt a sustainable approach to fitness that brings lasting transformation. I was once...",2023-06-04T20:59:32+0000,NaN,NaN,NaN


In [16]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 0 to 78
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   post_id                 79 non-null     object
 1   post_message            78 non-null     object
 2   post_created_time       79 non-null     object
 3   comment_created_time    69 non-null     object
 4   parent_comment_message  69 non-null     object
 5   comment_id              69 non-null     object
dtypes: object(6)
memory usage: 4.3+ KB


## Selecting a Subset of Comments for Analysis

In [ ]:
def process_posts(df_raw):
    """
    Return the unique non-null parent_comment_message values and the associated post_message values.
    """
    df = df_raw.copy()
    df = df[['parent_comment_message', 'post_message']].drop_duplicates()
    df = df[df['parent_comment_message'].notnull()]
    df = df[df['post_message'].notnull()]

    # Sort the dataframe by 'post_message' length in descending order
    df.sort_values(by='parent_comment_message', key=lambda x: x.str.len(), ascending=False, inplace=True)
    df.drop_duplicates(subset='post_message', keep='first', inplace=True)

    results = pd.DataFrame(
        df.apply(lambda row: f"YOU: {row['post_message']}\n\nMEMBER: {row['parent_comment_message']}", axis=1).unique(),
        columns=['text'])
    
    results['text'] = results['text'].str.replace('Amanda', 'Amy')
    results['text'] = results['text'].str.replace('amanda', 'amy')
    print(f'Shape of processed DataFrame containing Facebook group posts and comments: {results.shape}')
    return results

input_df = process_posts(merged)
input_df


Shape of processed DataFrame containing Facebook group posts and comments: (19, 1)


,text
0,"You: How I stay bikini confident year round 👙\n\nFirst off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel.\n\nSo let me tell you a little story…\n\nOnce upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. \n\nI thought having abs and being lean were the definition of the perfect body.\n\nAnd I embarked on a journey to achieve ‘perfection’.\n\nI ended up exhausted, overwhelmed and disappointed 😔 \n\nI was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life.\n\nAfter a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). \n\nSo what has this got to do with me being happy in a bikini? 🤣\n\nSeeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. \n\nMy mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. \n\nAnd (for anyone who read the whole copy) that is how I am now bikini confident year round 👙\n\nMember: Mind set is everything! Recently in Thailand, our last day. 12 hrs to kill before the flight, wanted to go for a swim, but didn’t want our baggage to weigh over the 25kg with wet swimmers (LOTS of shopping was done lol) so my friend and I stripped off to our bra and undies walked the 5 metres across the decking and went swimming! 😂🤣 12 mths ago before meeting u, I would never have dreamed of doing this 🤣😂🤣 you only have one body, just learn to love it! ❤️\nPS you look flipping amazballz!"
1,"You: Do it for you ❤️There will always be critics - silence them ✌️#beunapologeticallyyou\n\nMember: This comment makes me want to throat 👊🏻👊🏻👊🏻👊🏻\nI’m so glad you have broad shoulders, cause I think you look AMAZBALLZ!! I’d love to have a figure like yours, but I’m a wee bit busy, so I’ll settle for the one you helped me get!! ❤️😉 If someone is doing things to please another, what a shallow life they live! Love your response stay positive and keep smiling that gorgeous smile of yours! 💪🏻❤️🦵🏻"
2,"You: Hey @everyone Throw me some questions for our live training tomorrow? Nutrition, exercise, weight loss etc. anything you need to know - that’s what I’m here for\n\nMember: Need some tips for tightening tummy … lost 7kg in 3 months but now got stuck in same weight for more than 2 weeks … what do I need to change??"
3,You: @everyone can you please send me some questions for our live training Thursday! Last weeks questions were great! Remember I’m trying to help you all as much as possible so use this opportunity 👌\n\nMember: Did my measurements today and although my chest and hips seem to be coming down my stomach seems to be going up 😳 what am I doing wrong.
4,"You: Menopause - weights or cardio, 3 sessions a week - what to do, macros and calories - what changes, measurements\n\nMember: Thanks a million. You covered my question perfectly. I couldn't care less about calories but am trying to get the old bod healthy 🙂"
5,You: Greetings team from Airlie Beach 👋\n\nReminder there will be no weekly live training next week as I am on holidays 🙏\n\nI’m curious - have you any holidays planned this year?\n\nMember: You are less than 2 hours away from where I am thats so cool I love airlie beach just never find the time to go there anymore
6,"You: ‘I’ll never get results like her’\n\nDo you find yourself thinking or saying that? If you do - you’re not alone!!!!\n\nThis is something I hear REGULARL

# Seek ChatGPT's Wisdom using LangChain

Here are the prompt template I used to generate my final responses:

Task | Prompt Template
--- | ---
Sentiment | You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health conditions or injuries, or express dissatisfaction with their current fitness. Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, Determine the sentiment of the member comment as "positive", "neutral", or "negative". ```{text}``` If you are not sure of the answer, return "?"
Emotion | You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health conditions or injuries, or express dissatisfaction with their current fitness. Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, Determine the main emotion expressed in the member comment. Return your response as a single word. ```{text}``` If you are not sure of the answer, return "?"
Respond | You are the owner of a personal training business. You use a Facebook group to as a lead generation tool. You write posts in the group to engage with members and position yourself as a coach who can help them achieve their fitness goals. You aim to respond quickly to comments by group members if they request information, express a desire to improve their fitness, have health conditions or injuries, or express dissatisfaction with their current fitness. Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, Indicate whether you need to quickly respond to the message or not using "yes" or "no". ```{text}``` If you are not sure of the answer, return "?"

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import os
os.getenv('OPENAI_API_KEY')

def create_LLMChain(template, input_variables, output_key, llm=OpenAI(temperature=0, max_tokens=100)):
    """
    Create an LLMChain object.
    Parameters:
        - template (str): The template for the prompt, i.e. the value of the `template` 
            parameter in the PromptTemplate class.
        - input_variables (List[str]) 
        - output_key (str): The key of the output variable.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - LLMChain object
    """
    prompt_template = PromptTemplate(
        template=template, input_variables=input_variables)
    chain = LLMChain(
        llm=llm, prompt=prompt_template, output_key=output_key)
    return chain, prompt_template.template

def analyze_text_sentiment(text, llm=OpenAI(temperature=0, max_tokens=200)):
    """
    Analyze the sentiment of a text.
    Parameters:
        - text (str): The text to analyze.
        - llm (OpenAI object): Default is OpenAI()

    Returns:
        - sentiment (str): The sentiment of the text.
    """
    
    # openai.api_key = os.getenv('api_openai_p')
    template_root = """
    You are the owner of a personal training business. You use a Facebook group to as a lead generation tool.
    You write posts in the group to engage with members and position yourself as \
    a coach who can help them achieve their fitness goals.
    You aim to respond quickly to comments by group members if they request information, \
    express a desire to improve their fitness, have health conditions or injuries, \
    or express dissatisfaction with their current fitness.
    Below, delimited by triple back ticks, is a post you wrote in the group and a comment from a member:, 
    """
    templates = {
        'sentiment': 'Determine the sentiment of the member comment as "positive", "neutral", or "negative".',
        'emotions': 'Determine the main emotion expressed in the member comment. Return your response as a single word.',
        'respond': 'Indicate whether you need to quickly respond to the message or not using "yes" or "no".',
    }
    template_end = 'If you are not sure of the answer, return "?"'
    chain_dict = {}
    prompt_dict = {}
    for template in templates:
        chain_dict[template], prompt_dict[template] = create_LLMChain(
            template_root+templates[template]+' ```{text}``` '+template_end, 
            input_variables=['text'], output_key=template,
            llm=llm)
    overall_chain = SequentialChain(
        chains = [chain for chain in chain_dict.values()], 
        input_variables = ['text' for chain in chain_dict.values()],
        output_variables = list(chain_dict.keys()), verbose=True
    )
    response_dict = overall_chain(text)
    return response_dict, prompt_dict

def batch_analyze_text_sentiment_from_df(messages_df, key='text'):
    response_dict = dict()
    prompt_dict = dict()
    messages_list = messages_df[key].tolist()
    failures = 0
    for iteration_id, message in enumerate(messages_list):
        # print(iteration_id, message)
        try:
            response_dict[iteration_id], prompt_dict[iteration_id] = analyze_text_sentiment(message)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", file, ":", error)
            print('\t...Error making chatbot request')
            failures += 1
            if failures > 2:
                print('\t...Too many failures, stopping')
                break
    return response_dict, prompt_dict

def create_df_from_response(response):
    """
    Convert the response into a dataframe and strip white spaces at the start and end of text.
    """
    df = pd.DataFrame(response).transpose()
    for column in df.columns:
        df[column] = df[column].str.strip().str.replace('\n', ' ')
        df[column] = df[column].str.strip().str.replace('YOU:', '[YOU]')
        df[column] = df[column].str.strip().str.replace('MEMBER:', '[MEMBER]')
    return df

prompt_dict = dict()
message_dict = dict()
response_dict = dict()

iteration_id = 2

response_dict[iteration_id], prompt_dict[iteration_id] = batch_analyze_text_sentiment_from_df(input_df)

Shape of processed DataFrame containing Facebook group posts and comments: (19, 1)


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


In [ ]:
create_df_from_response(response_dict[iteration_id])

,text,sentiment,emotions,respond
0,"[YOU] How I stay bikini confident year round 👙 First off I need to tell you - being bikini confident has actually NOTHING to do with how you look and everything to do with how you feel. So let me tell you a little story… Once upon a time I was a self conscious, controlling perfectionist who believed that being skinny and lean would make me happy. I thought having abs and being lean were the definition of the perfect body. And I embarked on a journey to achieve ‘perfection’. I ended up exhausted, overwhelmed and disappointed 😔 I was unhappy - in a lot of elements of my life - so I decided to seek the help of a life coach. This was hard for me - as I’m a very strong independent woman so asking for help wasn’t easy. But let me tell you - hands down this changed my life. After a very difficult time in my life, I chose to work with a counsellor (for a TOTALLY different reason). So what has this got to do with me being happy in a bikini? 🤣 Seeking help from these people - changed my whole outlook on life. They helped me look internally and forced me to uncover some things I didn’t even know about myself. My mindset towards myself changed. I stopped focusing on aesthetics as a goal and started working on performance. My relationship with food did a 360 and I started viewing food as fuel instead of calories in versus calories out. I started practising gratitude and meditating. I prioritised sleep instead of ‘grinding’. And (for anyone who read the whole copy) that is how I am now bikini confident year round 👙 [MEMBER] Mind set is everything! Recently in Thailand, our last day. 12 hrs to kill before the flight, wanted to go for a swim, but didn’t want our baggage to weigh over the 25kg with wet swimmers (LOTS of shopping was done lol) so my friend and I stripped off to our bra and undies walked the 5 metres across the decking and went swimming! 😂🤣 12 mths ago before meeting u, I would never have dreamed of doing this 🤣😂🤣 you only have one body, just learn to love it! ❤️ PS you look flipping amazballz!",Positive,Empathy,No
1,"[YOU] Do it for you ❤️There will always be critics - silence them ✌️#beunapologeticallyyou [MEMBER] This comment makes me want to throat 👊🏻👊🏻👊🏻👊🏻 I’m so glad you have broad shoulders, cause I think you look AMAZBALLZ!! I’d love to have a figure like yours, but I’m a wee bit busy, so I’ll settle for the one you helped me get!! ❤️😉 If someone is doing things to please another, what a shallow life they live! Love your response stay positive and keep smiling that gorgeous smile of yours! 💪🏻❤️🦵🏻",Positive,Encouragement,No
2,"[YOU] Hey @everyone Throw me some questions for our live training tomorrow? Nutrition, exercise, weight loss etc. anything you need to know - that’s what I’m here for [MEMBER] Need some tips for tightening tummy … lost 7kg in 3 months but now got stuck in same weight for more than 2 weeks … what do I need to change??",Positive,Desire,Yes
3,[YOU] @everyone can you please send me some questions for our live training Thursday! Last weeks questions were great! Remember I’m trying to help you all as much as possible so use this opportunity 👌 [MEMBER] Did my measurements today and although my chest and hips seem to be coming down my stomach seems to be going up 😳 what am I doing wrong.,Positive,Frustration,Yes
4,"[YOU] Menopause - weights or cardio, 3 sessions a week - what to do, macros and calories - what changes, measurements [MEMBER] Thanks a million. You covered my question perfectly. I couldn't care less about calories but am trying to get the old bod healthy 🙂",Positive,Thanks,No
5,[YOU] Greetings team from Airlie Beach 👋 Reminder there will be no weekly live training next week as I am on holidays 🙏 I’m curious - have you any holidays planned this year? [MEMBER] You are less than 2 hours away from where I am thats so cool I love airlie beach just never find the time to go there anymore,Positive,Disappointment,No
6,"[YOU] ‘I’ll never get results like her’ Do you f

# *End of Page*